<a href="https://colab.research.google.com/github/Hassanm256/CS3120/blob/main/HR_employee_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# EDA & Preprocessing: Employee Attrition Analysis
# Intermediate Level - Industry Best Practices

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configure visualization defaults
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================
# STEP 0: Create Realistic Employee Dataset
# ============================================

def create_employee_dataset(n_employees=1500):
    """
    Creates a realistic employee dataset with common data quality issues
    """
    np.random.seed(42)

    # Generate employee data with realistic patterns
    data = {
        'EmployeeID': [f'EMP{str(i).zfill(4)}' for i in range(1, n_employees+1)],

        # Demographics
        'Age': np.random.normal(38, 10, n_employees).astype(int),
        'Gender': np.random.choice(['Male', 'Female', 'M', 'F', 'Other', np.nan],
                                  n_employees, p=[0.4, 0.4, 0.05, 0.05, 0.05, 0.05]),

        # Job Info - with inconsistent entries
        'Department': np.random.choice(['Sales', 'sales', 'SALES', 'Engineering', 'engineering',
                                       'HR', 'Human Resources', 'Marketing', 'Mktg', np.nan],
                                      n_employees),
        'JobRole': np.random.choice(['Manager', 'Senior Manager', 'Sr. Manager', 'Analyst',
                                    'Senior Analyst', 'Developer', 'Sr Developer', np.nan],
                                   n_employees),

        # Dates with various formats
        'HireDate': [datetime.now() - timedelta(days=np.random.randint(1, 3650))
                    for _ in range(n_employees)],

        # Compensation - different scales and missing patterns
        'Salary': np.random.lognormal(11, 0.5, n_employees),  # Log-normal distribution
        'StockOptions': np.random.choice([0, 1, 2, 3, np.nan], n_employees,
                                        p=[0.4, 0.25, 0.15, 0.1, 0.1]),

        # Performance metrics
        'PerformanceRating': np.random.choice([1, 2, 3, 4, 5, '3', '4', 'Good', 'Excellent', np.nan],
                                            n_employees),
        'ProjectsCompleted': np.random.poisson(5, n_employees),

        # Work-life balance indicators
        'WorkHoursPerWeek': np.random.normal(45, 8, n_employees),
        'BusinessTravel': np.random.choice(['No', 'Rarely', 'Frequently', 'No Travel', np.nan],
                                         n_employees),
        'DistanceFromHome': np.abs(np.random.normal(15, 10, n_employees)),

        # Satisfaction scores (different scales mixed)
        'JobSatisfaction': np.random.choice([1, 2, 3, 4, 5,  # 1-5 scale
                                           10, 20, 30, 40, 50,  # 10-50 scale
                                           np.nan], n_employees),

        # Target variable
        'Attrition': np.random.choice(['Yes', 'No', 'Y', 'N', 1, 0], n_employees,
                                     p=[0.1, 0.7, 0.05, 0.1, 0.025, 0.025])
    }

    df = pd.DataFrame(data)

    # Add realistic missing patterns
    # Young employees less likely to have stock options
    mask = (df['Age'] < 25) & (np.random.random(n_employees) < 0.6)
    df.loc[mask, 'StockOptions'] = np.nan

    # High performers might not report satisfaction (too busy)
    mask = (df['PerformanceRating'].isin(['5', 'Excellent'])) & (np.random.random(n_employees) < 0.3)
    df.loc[mask, 'JobSatisfaction'] = np.nan

    # Add some outliers
    df.loc[np.random.choice(df.index, 3), 'Age'] = [65, 70, 18]
    df.loc[np.random.choice(df.index, 5), 'WorkHoursPerWeek'] = [80, 85, 90, 10, 5]

    # Add duplicates
    dup_indices = np.random.choice(df.index, 10)
    df = pd.concat([df, df.loc[dup_indices]], ignore_index=True)

    return df

# Create the dataset
print("Creating employee dataset...")
df = create_employee_dataset()
print(f"Dataset created with {len(df)} records and {len(df.columns)} features")

# ============================================
# CHECKPOINT 1: Initial Data Inspection
# ============================================

print("\n" + "="*60)
print("CHECKPOINT 1: What do we see?")
print("="*60)

# Quick look at the data
print("\nFirst 5 rows:")
print(df.head())

print("\nData types:")
print(df.dtypes)

print("\nBasic info:")
print(f"Shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# ============================================
# 💡 MENTOR QUESTION 1
# ============================================
print("\n" + "📊 MENTOR QUESTION 1: Looking at this data, what issues do you notice?")
print("Before we continue, try to identify at least 3 data quality issues.")
print("Hint: Look at Gender, Department, PerformanceRating, and Attrition columns...")
print("\nThink about:")
print("- Inconsistent formatting")
print("- Mixed data types")
print("- Potential duplicates")
print("- Missing values")

# Let me show you what to look for:
print("\n" + "-"*40)
print("Example - Gender column values:")
print(df['Gender'].value_counts(dropna=False))

print("\nExample - Attrition column values:")
print(df['Attrition'].value_counts(dropna=False))

Creating employee dataset...
Dataset created with 1510 records and 15 features

CHECKPOINT 1: What do we see?

First 5 rows:
  EmployeeID  Age  Gender       Department       JobRole  \
0    EMP0001   42    Male      Engineering  Sr Developer   
1    EMP0002   36       M            SALES     Developer   
2    EMP0003   44  Female  Human Resources   Sr. Manager   
3    EMP0004   53    Male        Marketing       Analyst   
4    EMP0005   35    Male            sales     Developer   

                    HireDate        Salary  StockOptions PerformanceRating  \
0 2019-05-05 20:57:21.428816  94610.618149           0.0                 4   
1 2019-12-07 20:57:21.428854  57348.750330           0.0              Good   
2 2022-05-15 20:57:21.428865  85887.231020           2.0                 3   
3 2020-10-30 20:57:21.428872  51863.147730           0.0                 1   
4 2021-10-11 20:57:21.428879  65212.995120           0.0                 4   

   ProjectsCompleted  WorkHoursPerWeek Busine

In [2]:
# Create a copy first (why is this important?)
df_clean = df.copy()

# Your code here to fix Gender
# Hint: Consider using .map() or .replace()

In [3]:
df['Gender'].value_counts(dropna=False)

,count
Gender,
Female,611
Male,607
M,80
nan,78
F,75
Other,59


In [4]:
Gender_mapping = { 'M': 'Male',
                  'F': 'Female',
                   }
df['Gender'] = df['Gender'].replace(Gender_mapping)

In [5]:
df['Gender'].value_counts(dropna=False)

,count
Gender,
Male,687
Female,686
nan,78
Other,59


In [6]:
df[df['Gender'] == 'Other'].head()

,EmployeeID,Age,Gender,Department,JobRole,HireDate,Salary,StockOptions,PerformanceRating,ProjectsCompleted,WorkHoursPerWeek,BusinessTravel,DistanceFromHome,JobSatisfaction,Attrition
19,EMP0020,23,Other,Human Resources,Developer,2022-06-11 20:57:21.428983,62096.242974,0.0,2,6,45.851635,No Travel,11.663004,10.0,1
39,EMP0040,39,Other,HR,nan,2024-05-25 20:57:21.429129,30527.868294,NaN,5,5,48.558004,No,2.359076,NaN,No
66,EMP0067,37,Other,Engineering,Developer,2022-10-29 20:57:21.429332,53099.911629,NaN,3,5,42.857967,Frequently,22.513367,50.0,N
76,EMP0077,38,Other,engineering,Developer,2021-06-01 20:57:21.429394,76005.590353,0.0,4,5,48.324081,No Travel,13.173489,40.0,No
86,EMP0087,47,Other,Human Resources,Senior Manager,2016-12-23 20:57:21.429460,66339.062987,1.0,1,3,43.253567,No,2.329488,2.0,No


In [10]:
df[df['Gender'] == 'Other']['Department'].value_counts()


,count
Department,
Engineering,11
Marketing,8
sales,7
Human Resources,7
HR,5
engineering,5
nan,5
Mktg,5
Sales,3


In [11]:
df[df['Gender'] == 'Other']['Age'].describe()

,Age
count,59.000000
mean,36.898305
std,7.606021
min,21.000000
25%,32.000000
50%,38.000000
75%,41.000000
max,58.000000


In [12]:
dept_gender = pd.crosstab(df['Department'], df['Gender'], normalize='index') * 100
print(dept_gender)

Gender              Female       Male     Other       nan
Department                                               
Engineering      42.038217  45.859873  7.006369  5.095541
HR               46.242775  47.976879  2.890173  2.890173
Human Resources  43.859649  45.614035  4.093567  6.432749
Marketing        41.935484  47.096774  5.161290  5.806452
Mktg             44.520548  45.890411  3.424658  6.164384
SALES            49.006623  41.721854  1.986755  7.284768
Sales            42.537313  50.746269  2.238806  4.477612
engineering      53.435115  41.221374  3.816794  1.526718
nan              49.350649  42.207792  3.246753  5.194805
sales            42.028986  46.376812  5.072464  6.521739


In [13]:
# Option 1: Keep as is
# (no change needed)

# Option 2: Rename for clarity
df['Gender'] = df['Gender'].replace({'Other': 'Non-binary/Other'})

In [19]:
#fill missing values with Unknown
df['Gender'].fillna('Unkown')
df['Gender'].value_counts()

,count
Gender,
Male,687
Female,686
nan,78
Non-binary/Other,59


In [20]:
# Confirm no missing values
print(f"Missing values: {df['Gender'].isnull().sum()}")

# See final distribution
print(df['Gender'].value_counts())

Missing values: 0
Gender
Male                687
Female              686
nan                  78
Non-binary/Other     59
Name: count, dtype: int64


In [21]:
# First, let's check what type of "missing" values we actually have
print("Unique values:", df['Gender'].unique())

# Fix both actual NaN and string 'nan'
df['Gender'] = df['Gender'].replace('nan', 'Unknown')  # Fix string 'nan'
df['Gender'] = df['Gender'].fillna('Unknown')          # Fix actual NaN

# Verify the fix
print("Missing values:", df['Gender'].isnull().sum())
print(df['Gender'].value_counts())

Unique values: ['Male' 'Female' 'nan' 'Non-binary/Other']
Missing values: 0
Gender
Male                687
Female              686
Unknown              78
Non-binary/Other     59
Name: count, dtype: int64


In [22]:
# See what department issues we have
print("Department value counts:")
print(df['Department'].value_counts(dropna=False))

Department value counts:
Department
HR                 173
Human Resources    171
Engineering        157
Marketing          155
nan                154
SALES              151
Mktg               146
sales              138
Sales              134
engineering        131
Name: count, dtype: int64


In [23]:
# See what attrition issues we have
print("Attrition value counts:")
print(df['Attrition'].value_counts(dropna=False))

Attrition value counts:
Attrition
No     1050
Yes     170
N       143
Y        73
1        38
0        36
Name: count, dtype: int64


In [27]:
Department_Mapping = {
    'HR': 'HR',
    'Human Resources': 'HR',
    'SALES': 'Sales',
    'sales': 'Sales',
    'Sales': 'Sales',
    'Engineering': 'Engineering',
    'engineering': 'Engineering',
    'Marketing': 'Marketing',
    'Mktg': 'Marketing'

}
df['Department'] = df['Department'].replace('nan', 'Unkown')

df['Department'] = df['Department'].fillna('Unkown')

print("Department after cleaning:")
print(df['Department'].value_counts())

Department after cleaning:
Department
Sales          423
HR             344
Marketing      301
Engineering    288
Unkown         154
Name: count, dtype: int64


In [29]:
attrition_mapping = {
    'Yes': 'Y',
    'Y': 'Y',
    'N': 'N',
    'No': 'N',
    'n': 'N',
    'y': 'Y',
    '1': 'Y',
    '0': 'N'
}
df['Attrition'] = df['Attrition'].replace(attrition_mapping)
print(df['Attrition'].value_counts())

Attrition
N    1229
Y     281
Name: count, dtype: int64


In [30]:
# Final verification - check for any remaining issues
print("=== FINAL DATA CLEANING SUMMARY ===")
print(f"Total records: {len(df)}")
print()

print("Gender distribution:")
print(df['Gender'].value_counts())
print(f"Missing values: {df['Gender'].isnull().sum()}")
print()

print("Department distribution:")
print(df['Department'].value_counts())
print(f"Missing values: {df['Department'].isnull().sum()}")
print()

print("Attrition distribution:")
print(df['Attrition'].value_counts())
print(f"Missing values: {df['Attrition'].isnull().sum()}")

=== FINAL DATA CLEANING SUMMARY ===
Total records: 1510

Gender distribution:
Gender
Male                687
Female              686
Unknown              78
Non-binary/Other     59
Name: count, dtype: int64
Missing values: 0

Department distribution:
Department
Sales          423
HR             344
Marketing      301
Engineering    288
Unkown         154
Name: count, dtype: int64
Missing values: 0

Attrition distribution:
Attrition
N    1229
Y     281
Name: count, dtype: int64
Missing values: 0


In [31]:
# See all column names
print("All columns in the dataset:")
print(df.columns.tolist())
print(f"\nTotal columns: {len(df.columns)}")

All columns in the dataset:
['EmployeeID', 'Age', 'Gender', 'Department', 'JobRole', 'HireDate', 'Salary', 'StockOptions', 'PerformanceRating', 'ProjectsCompleted', 'WorkHoursPerWeek', 'BusinessTravel', 'DistanceFromHome', 'JobSatisfaction', 'Attrition']

Total columns: 15


In [32]:
# Quick overview of all columns
df.info()

# Or look at data types
df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1510 entries, 0 to 1509
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   EmployeeID         1510 non-null   object        
 1   Age                1510 non-null   int64         
 2   Gender             1510 non-null   object        
 3   Department         1510 non-null   object        
 4   JobRole            1510 non-null   object        
 5   HireDate           1510 non-null   datetime64[ns]
 6   Salary             1510 non-null   float64       
 7   StockOptions       1289 non-null   float64       
 8   PerformanceRating  1510 non-null   object        
 9   ProjectsCompleted  1510 non-null   int64         
 10  WorkHoursPerWeek   1510 non-null   float64       
 11  BusinessTravel     1510 non-null   object        
 12  DistanceFromHome   1510 non-null   float64       
 13  JobSatisfaction    1272 non-null   float64       
 14  Attritio

,0
EmployeeID,object
Age,int64
Gender,object
Department,object
JobRole,object
HireDate,datetime64[ns]
Salary,float64
StockOptions,float64
PerformanceRating,object
ProjectsCompleted,int64


In [33]:
# Check PerformanceRating issues
print("PerformanceRating value counts:")
print(df['PerformanceRating'].value_counts(dropna=False))

print("\nUnique values:")
print(df['PerformanceRating'].unique())

print("\nData type:")
print(df['PerformanceRating'].dtype)

PerformanceRating value counts:
PerformanceRating
4            318
3            288
Excellent    167
Good         157
2            151
nan          147
5            146
1            136
Name: count, dtype: int64

Unique values:
['4' 'Good' '3' '1' '5' 'nan' '2' 'Excellent']

Data type:
object


In [41]:
# We need to go back to the previous step before filling
# Let's re-do the standardization but NOT fill missing values

# Re-apply the standardization mapping
performance_mapping = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    'Good': 4,
    'Excellent': 5
}

# Re-apply mapping (assuming you still have the original mixed data)
df['PerformanceRating'] = df['PerformanceRating'].replace(performance_mapping)

# Handle missing values as "Unknown" instead of median
df['PerformanceRating'] = df['PerformanceRating'].replace('nan', 'Unknown')
df['PerformanceRating'] = df['PerformanceRating'].fillna('Unknown')

# Check results
print("Final PerformanceRating with Unknown category:")
print(df['PerformanceRating'].value_counts())

Final PerformanceRating with Unknown category:
PerformanceRating
4.0    622
5.0    313
3.0    288
2.0    151
1.0    136
Name: count, dtype: int64


In [42]:
# Since we filled 147 missing values with 4.0, we need to subtract them back
# Current 4.0 count is 622, but it should be 475 (original 4.0 + Good)
# So we need to convert 147 of the 4.0 values back to "Unknown"

# Let's identify which records originally had missing performance ratings
# First, let's see if we can identify them another way

# Check current value counts
print("Current distribution:")
print(df['PerformanceRating'].value_counts())

# We know 622 - 475 = 147 records that should be "Unknown"
# Let's randomly select 147 records with value 4.0 and change them to "Unknown"

import numpy as np
np.random.seed(42)  # For reproducibility

# Find indices where PerformanceRating is 4.0
four_indices = df[df['PerformanceRating'] == 4.0].index

# Randomly select 147 of them to change to Unknown
unknown_indices = np.random.choice(four_indices, size=147, replace=False)

# Change these to Unknown
df.loc[unknown_indices, 'PerformanceRating'] = 'Unknown'

# Check results
print("\nAfter converting back to Unknown:")
print(df['PerformanceRating'].value_counts())

Current distribution:
PerformanceRating
4.0    622
5.0    313
3.0    288
2.0    151
1.0    136
Name: count, dtype: int64

After converting back to Unknown:
PerformanceRating
4.0        475
5.0        313
3.0        288
2.0        151
Unknown    147
1.0        136
Name: count, dtype: int64


In [44]:
# Quick peek at remaining issues
print("JobRole values:")
print(df['JobRole'].value_counts())

JobRole values:
JobRole
Senior Manager    221
Sr Developer      196
Senior Analyst    192
Developer         189
Manager           189
Analyst           189
nan               181
Sr. Manager       153
Name: count, dtype: int64


In [46]:
# Check JobRole issues
print("JobRole value counts:")
print(df['JobRole'].value_counts(dropna=False))



JobRole value counts:
JobRole
Senior Manager    221
Sr Developer      196
Senior Analyst    192
Developer         189
Manager           189
Analyst           189
nan               181
Sr. Manager       153
Name: count, dtype: int64


In [47]:
# JobRole standardization mapping
jobrole_mapping = {
    'Manager': 'Manager',
    'Senior Manager': 'Senior Manager',
    'Sr. Manager': 'Senior Manager',    # Combine with Senior Manager
    'Analyst': 'Analyst',
    'Senior Analyst': 'Senior Analyst',
    'Developer': 'Developer',
    'Sr Developer': 'Senior Developer'   # Standardize to Senior Developer
}

# Apply the mapping
df['JobRole'] = df['JobRole'].replace(jobrole_mapping)

# Handle missing values
df['JobRole'] = df['JobRole'].replace('nan', 'Unknown')

# Check results
print("JobRole after cleaning:")
print(df['JobRole'].value_counts())

JobRole after cleaning:
JobRole
Senior Manager      374
Senior Developer    196
Senior Analyst      192
Developer           189
Analyst             189
Manager             189
Unknown             181
Name: count, dtype: int64


In [48]:
# Check what happened - let's see if the mapping worked
print("Let's check the original 'Sr Developer' entries:")
print("Current unique JobRole values:", df['JobRole'].unique())

# The issue might be that 'Sr Developer' didn't map correctly
# Let's fix this manually:
df['JobRole'] = df['JobRole'].replace('Sr Developer', 'Senior Developer')

# Also combine the remaining Developer entries:
print("Before final fix:")
print(df['JobRole'].value_counts())

# Now check results
print("\nAfter fixing Sr Developer:")
df['JobRole'] = df['JobRole'].replace('Developer', 'Senior Developer')
print(df['JobRole'].value_counts())

Let's check the original 'Sr Developer' entries:
Current unique JobRole values: ['Senior Developer' 'Developer' 'Senior Manager' 'Analyst' 'Unknown'
 'Senior Analyst' 'Manager']
Before final fix:
JobRole
Senior Manager      374
Senior Developer    196
Senior Analyst      192
Developer           189
Analyst             189
Manager             189
Unknown             181
Name: count, dtype: int64

After fixing Sr Developer:
JobRole
Senior Developer    385
Senior Manager      374
Senior Analyst      192
Analyst             189
Manager             189
Unknown             181
Name: count, dtype: int64


In [49]:
# Missing value summary
print("Missing values across all columns:")
missing_summary = df.isnull().sum()
missing_percentage = (missing_summary / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_summary,
    'Missing_Percentage': missing_percentage
}).sort_values('Missing_Count', ascending=False)

print(missing_df[missing_df['Missing_Count'] > 0])

Missing values across all columns:
                 Missing_Count  Missing_Percentage
JobSatisfaction            238           15.761589
StockOptions               221           14.635762


In [50]:
# Investigate missing patterns
print("=== MISSING VALUE PATTERN ANALYSIS ===")

# 1. Are missing values related to employee characteristics?
print("1. Missing StockOptions by Age:")
print("Young employees (<25) missing StockOptions:")
young_missing_stock = df[(df['Age'] < 25) & (df['StockOptions'].isnull())]
print(f"Count: {len(young_missing_stock)} out of {len(df[df['Age'] < 25])} young employees")

print("\n2. Missing JobSatisfaction by PerformanceRating:")
print("High performers missing JobSatisfaction:")
high_perf_missing_sat = df[(df['PerformanceRating'] == 5.0) & (df['JobSatisfaction'].isnull())]
print(f"Count: {len(high_perf_missing_sat)}")

print("\n3. Overall missing patterns:")
print("Employees missing BOTH StockOptions AND JobSatisfaction:")
both_missing = df[(df['StockOptions'].isnull()) & (df['JobSatisfaction'].isnull())]
print(f"Count: {len(both_missing)}")

=== MISSING VALUE PATTERN ANALYSIS ===
1. Missing StockOptions by Age:
Young employees (<25) missing StockOptions:
Count: 89 out of 134 young employees

2. Missing JobSatisfaction by PerformanceRating:
High performers missing JobSatisfaction:
Count: 109

3. Overall missing patterns:
Employees missing BOTH StockOptions AND JobSatisfaction:
Count: 28


In [51]:
# Create meaningful categories for missing values
print("=== CREATING MEANINGFUL MISSING CATEGORIES ===")

# For StockOptions: Fill based on employee profile
df['StockOptions_Clean'] = df['StockOptions'].copy()
df.loc[df['StockOptions'].isnull(), 'StockOptions_Clean'] = 'Not Eligible'

# For JobSatisfaction: Create "Not Reported" category
df['JobSatisfaction_Clean'] = df['JobSatisfaction'].copy()
df.loc[df['JobSatisfaction'].isnull(), 'JobSatisfaction_Clean'] = 'Not Reported'

# Check results
print("StockOptions distribution:")
print(df['StockOptions_Clean'].value_counts())
print("\nJobSatisfaction missing handled:")
print(f"Not Reported: {(df['JobSatisfaction_Clean'] == 'Not Reported').sum()}")

=== CREATING MEANINGFUL MISSING CATEGORIES ===
StockOptions distribution:
StockOptions_Clean
0.0             568
1.0             349
2.0             229
Not Eligible    221
3.0             143
Name: count, dtype: int64

JobSatisfaction missing handled:
Not Reported: 238


In [52]:
# Final verification
print("=== FINAL CLEAN DATASET SUMMARY ===")
print(f"Total records: {len(df)}")
print(f"Total columns: {len(df.columns)}")
print(f"Columns with missing values: {df.isnull().sum().sum()}")

=== FINAL CLEAN DATASET SUMMARY ===
Total records: 1510
Total columns: 17
Columns with missing values: 459


In [53]:
# Let's see exactly which columns still have missing values
print("Remaining missing values by column:")
missing_summary = df.isnull().sum()
missing_df = missing_summary[missing_summary > 0].sort_values(ascending=False)
print(missing_df)

print("\nColumn names and types:")
print(df.dtypes)

Remaining missing values by column:
JobSatisfaction    238
StockOptions       221
dtype: int64

Column names and types:
EmployeeID                       object
Age                               int64
Gender                           object
Department                       object
JobRole                          object
HireDate                 datetime64[ns]
Salary                          float64
StockOptions                    float64
PerformanceRating                object
ProjectsCompleted                 int64
WorkHoursPerWeek                float64
BusinessTravel                   object
DistanceFromHome                float64
JobSatisfaction                 float64
Attrition                        object
StockOptions_Clean               object
JobSatisfaction_Clean            object
dtype: object


In [54]:
# Verify our cleaned columns have no missing values
print("=== VERIFICATION OF CLEANED COLUMNS ===")
print("StockOptions_Clean missing:", df['StockOptions_Clean'].isnull().sum())
print("JobSatisfaction_Clean missing:", df['JobSatisfaction_Clean'].isnull().sum())

print("\n=== CHECKING OUR 'UNKNOWN' CATEGORIES ===")
print("Gender nulls:", df['Gender'].isnull().sum())
print("Department nulls:", df['Department'].isnull().sum())
print("JobRole nulls:", df['JobRole'].isnull().sum())
print("PerformanceRating nulls:", df['PerformanceRating'].isnull().sum())

print("\n=== CLEAN DATASET READY FOR ANALYSIS ===")
print("Use these cleaned columns:")
print("- StockOptions_Clean (instead of StockOptions)")
print("- JobSatisfaction_Clean (instead of JobSatisfaction)")

=== VERIFICATION OF CLEANED COLUMNS ===
StockOptions_Clean missing: 0
JobSatisfaction_Clean missing: 0

=== CHECKING OUR 'UNKNOWN' CATEGORIES ===
Gender nulls: 0
Department nulls: 0
JobRole nulls: 0
PerformanceRating nulls: 0

=== CLEAN DATASET READY FOR ANALYSIS ===
Use these cleaned columns:
- StockOptions_Clean (instead of StockOptions)
- JobSatisfaction_Clean (instead of JobSatisfaction)


In [55]:
# Get the big picture first
print("Overall attrition rate:")
print(df['Attrition'].value_counts(normalize=True) * 100)

Overall attrition rate:
Attrition
N    81.390728
Y    18.609272
Name: proportion, dtype: float64


# New Section